In [1]:
from IPython.display import display, Image
import os, warnings
import re
warnings.filterwarnings(action='ignore')
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver import ActionChains
import numpy as np
import pandas as pd

### 데이터 전처리
* 컬럼별 데이터 자료형 변환 및 데이터 정제
* Feature engineering 하기 (대륙명 데이터 추가, iso 국가 코드 flag icon에서 2자리 문자열 데이터로 변환)

In [28]:
data_df = pd.read_excel("./covid_df.xlsx")

In [29]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  221 non-null    int64  
 1   국가          221 non-null    object 
 2   확진자         221 non-null    object 
 3   사망자         221 non-null    object 
 4   완치          218 non-null    object 
 5   치명(%)       221 non-null    object 
 6   완치(%)       218 non-null    float64
 7   발생률         221 non-null    object 
 8   인구수         221 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 15.7+ KB


In [30]:
# 확진자 컬럼 데이터 정제

data_df["total_patients"] = data_df["확진자"].str.split("\n").str[0]
data_df["total_patients"] = data_df["total_patients"].apply(lambda x: x.replace(",", "")).astype(int)
data_df["total_patients"]

0      34434803
1      30028709
2      18056639
3       5760002
4       5381736
         ...   
216           4
217           4
218           3
219           2
220           1
Name: total_patients, Length: 221, dtype: int32

In [31]:
# 사망자 컬럼 데이터 정제

data_df["total_death"] = data_df["사망자"].str.split("\n").str[0]
data_df["total_death"] = data_df["total_death"].apply(lambda x: x.replace(",", "")).astype(int)
data_df["total_death"]

0      617875
1      390691
2      504897
3      110829
4       49293
        ...  
216         0
217         1
218         0
219         0
220         0
Name: total_death, Length: 221, dtype: int32

In [32]:
# Null 값이 있는 행 제거 (N/A 값이 있는 것은 제거)

data_df["total_recovered"] = data_df["완치"].str.split("\n").str[0]
data_df.dropna(inplace=True)
data_df["total_recovered"] = data_df["total_recovered"].apply(lambda x: x.replace(",", "")).astype(int)
print(data_df.shape)
data_df["total_recovered"]

(218, 12)


0      28817134
1      28994855
2      16388847
3       5575086
4       5242945
         ...   
216           4
217           3
218           3
219           2
220           1
Name: total_recovered, Length: 218, dtype: int32

In [33]:
# 치명율 데이터 정제

data_df["critical_rate"] = data_df["치명(%)"].apply(lambda x: x.replace("-", "0")).astype(float)
data_df["critical_rate"] = data_df["critical_rate"].apply(lambda x: x / 100)
data_df["critical_rate"]

0      0.018
1      0.013
2      0.028
3      0.019
4      0.009
       ...  
216    0.000
217    0.250
218    0.000
219    0.000
220    0.000
Name: critical_rate, Length: 218, dtype: float64

In [34]:
# 완치율 데이터 정제

data_df["recovery_rate"] = data_df["완치(%)"].astype(float)
data_df["recovery_rate"] = data_df["recovery_rate"].apply(lambda x: x / 100)
data_df["recovery_rate"]

0      0.837
1      0.966
2      0.908
3      0.968
4      0.974
       ...  
216    1.000
217    0.750
218    1.000
219    1.000
220    1.000
Name: recovery_rate, Length: 218, dtype: float64

In [35]:
# 100만명당 확진자 데이터 정제 (발생률)

data_df["patient_per_million"] = data_df["발생률"].apply(lambda x: x.replace(",", ""))
data_df["patient_per_million"] = data_df["patient_per_million"].apply(lambda x: x.replace("-", "0")).astype(int)
data_df["patient_per_million"]

0      104032
1       21760
2       84949
3       88244
4       63811
        ...  
216        68
217        13
218        15
219       329
220         9
Name: patient_per_million, Length: 218, dtype: int32

In [36]:
# 인구수 데이터 정제

data_df["total_population"] = data_df["인구수"].str.split("\n").str[0]
data_df["total_population"] = data_df["total_population"].apply(lambda x: x.replace("-", "0"))
data_df["total_population"] = data_df["total_population"].apply(lambda x: x.replace(",", "")).astype(int)
data_df["total_population"]

0       331002651
1      1380004385
2       212559417
3        65273511
4        84339067
          ...    
216         59190
217        307145
218        198414
219          6077
220        115023
Name: total_population, Length: 218, dtype: int32

In [37]:
# 국가별 코드 데이터 컬럼 분리하여 코드명과 국가명 컬럼 추가

data_df["국가"] = data_df["국가"].apply(lambda x: x.replace("*", "")) # 일본 국가명에는 *이 들어가 있어서 처리
data_df["country_code"] = data_df["국가"].str[-2:].astype(str)
data_df["country_name"] = data_df["국가"].str[:-2].astype(str)
data_df["country_name"]

0          미국
1          인도
2         브라질
3         프랑스
4          터키
        ...  
216     마셜 제도
217      바누아투
218       사모아
219    세인트헬레나
220    미크로네시아
Name: country_name, Length: 218, dtype: object

In [38]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218 entries, 0 to 220
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           218 non-null    int64  
 1   국가                   218 non-null    object 
 2   확진자                  218 non-null    object 
 3   사망자                  218 non-null    object 
 4   완치                   218 non-null    object 
 5   치명(%)                218 non-null    object 
 6   완치(%)                218 non-null    float64
 7   발생률                  218 non-null    object 
 8   인구수                  218 non-null    object 
 9   total_patients       218 non-null    int32  
 10  total_death          218 non-null    int32  
 11  total_recovered      218 non-null    int32  
 12  critical_rate        218 non-null    float64
 13  recovery_rate        218 non-null    float64
 14  patient_per_million  218 non-null    int32  
 15  total_population     218 non-null    int

In [39]:
df_for_analysis = data_df.drop(["Unnamed: 0", "국가", "확진자", "사망자", "완치", "치명(%)", "완치(%)", "발생률", "인구수"], axis=1)
df_for_analysis

,total_patients,total_death,total_recovered,critical_rate,recovery_rate,patient_per_million,total_population,country_code,country_name
0,34434803,617875,28817134,0.018,0.837,104032,331002651,🇺🇸,미국
1,30028709,390691,28994855,0.013,0.966,21760,1380004385,🇮🇳,인도
2,18056639,504897,16388847,0.028,0.908,84949,212559417,🇧🇷,브라질
3,5760002,110829,5575086,0.019,0.968,88244,65273511,🇫🇷,프랑스
4,5381736,49293,5242945,0.009,0.974,63811,84339067,🇹🇷,터키
...,...,...,...,...,...,...,...,...,...
216,4,0,4,0.000,1.000,68,59190,🇲🇭,마셜 제도
217,4,1,3,0.250,0.750,13,307145,🇻🇺,바누아투
218,3,0,3,0.000,1.000,15,198414,🇼🇸,사모아
219,2,0,2,0.000,1.000,329,6077,🇸🇭,세인트헬레나


In [40]:
df_for_analysis.describe()

,total_patients,total_death,total_recovered,critical_rate,recovery_rate,patient_per_million,total_population
count,2.180000e+02,218.000000,2.180000e+02,218.000000,218.000000,218.000000,2.180000e+02
mean,8.042192e+05,16876.454128,7.361992e+05,0.020798,0.874220,34949.458716,3.532511e+07
std,3.413364e+06,64797.903805,3.071127e+06,0.028321,0.181209,39656.057945,1.400734e+08
min,1.000000e+00,0.000000,1.000000e+00,0.000000,0.008000,0.000000,0.000000e+00
25%,6.191250e+03,91.000000,4.826000e+03,0.008250,0.868500,2419.500000,6.265000e+05
50%,4.889650e+04,835.000000,4.351100e+04,0.016000,0.949000,18465.500000,5.821272e+06
75%,3.524472e+05,5315.250000,3.279708e+05,0.024000,0.976000,62489.250000,2.321589e+07
max,3.443480e+07,617875.000000,2.899486e+07,0.250000,1.000000,179434.000000,1.439324e+09


In [41]:
iso_df = pd.read_excel("./iso_country_code.xlsx").rename(columns={"Unnamed: 3": "country_name", "Unnamed: 2": "country_iso_code"}, inplace=False)
iso_df = iso_df[["country_iso_code", "country_name"]]
iso_df = iso_df.drop([0, 1, 2], axis=0)
iso_df

,country_iso_code,country_name
3,GH,가나
4,GA,가봉
5,GY,가이아나
6,GM,감비아
7,GG,건지 섬
...,...,...
243,PH,필리핀
244,PN,핏케언 제도
245,HM,허드 맥도널드 제도
246,HU,헝가리


In [42]:
df_with_iso = pd.merge(df_for_analysis, iso_df, how='outer', on='country_name')
df_with_iso.dropna(subset=['country_code'], inplace=True)
df_with_iso.drop("country_code", axis=1, inplace=True)
df_with_iso

,total_patients,total_death,total_recovered,critical_rate,recovery_rate,patient_per_million,total_population,country_name,country_iso_code
0,34434803.0,617875.0,28817134.0,0.018,0.837,104032.0,3.310027e+08,미국,US
1,30028709.0,390691.0,28994855.0,0.013,0.966,21760.0,1.380004e+09,인도,IN
2,18056639.0,504897.0,16388847.0,0.028,0.908,84949.0,2.125594e+08,브라질,BR
3,5760002.0,110829.0,5575086.0,0.019,0.968,88244.0,6.527351e+07,프랑스,FR
4,5381736.0,49293.0,5242945.0,0.009,0.974,63811.0,8.433907e+07,터키,TR
...,...,...,...,...,...,...,...,...,...
213,4.0,0.0,4.0,0.000,1.000,68.0,5.919000e+04,마셜 제도,MH
214,4.0,1.0,3.0,0.250,0.750,13.0,3.071450e+05,바누아투,VU
215,3.0,0.0,3.0,0.000,1.000,15.0,1.984140e+05,사모아,WS
216,2.0,0.0,2.0,0.000,1.000,329.0,6.077000e+03,세인트헬레나,SH


In [43]:
continent_df = pd.read_csv("./country-and-continent-codes-list-csv.csv", encoding="utf-8")
continent_df = continent_df[["Continent_Name", "Two_Letter_Country_Code"]]
continent_df.rename(columns = {"Two_Letter_Country_Code": "country_iso_code", "Continent_Name": "continent_name"}, inplace=True)
continent_df

,continent_name,country_iso_code
0,Asia,AF
1,Europe,AL
2,Antarctica,AQ
3,Africa,DZ
4,Oceania,AS
...,...,...
257,Africa,ZM
258,Oceania,XX
259,Asia,XE
260,Asia,XD


In [44]:
final_df = pd.merge(df_with_iso, continent_df, how="left", left_on="country_iso_code", right_on="country_iso_code")
final_df.dropna(subset=["country_name"], inplace=True)
print(final_df.info())
print(final_df.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225 entries, 0 to 224
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   total_patients       225 non-null    float64
 1   total_death          225 non-null    float64
 2   total_recovered      225 non-null    float64
 3   critical_rate        225 non-null    float64
 4   recovery_rate        225 non-null    float64
 5   patient_per_million  225 non-null    float64
 6   total_population     225 non-null    float64
 7   country_name         225 non-null    object 
 8   country_iso_code     202 non-null    object 
 9   continent_name       225 non-null    object 
dtypes: float64(7), object(3)
memory usage: 19.3+ KB
None
(225, 10)


In [46]:
# 아시아와 유럽 두 대륙에 모두 속해 있는 중복값 처리 (7개 국가)

turkey_index = (final_df["country_name"] == "터키") & (final_df["continent_name"] == "Europe")
final_df.drop(final_df[turkey_index].index, axis=0, inplace=True)

russia_index = (final_df["country_name"] == "러시아") & (final_df["continent_name"] == "Asia")
final_df.drop(final_df[russia_index].index, axis=0, inplace=True)

georgia_index = (final_df["country_name"] == "조지아") & (final_df["continent_name"] == "Europe")
final_df.drop(final_df[georgia_index].index, axis=0, inplace=True)

kaza_index = (final_df["country_name"] == "카자흐스탄") & (final_df["continent_name"] == "Europe")
final_df.drop(final_df[kaza_index].index, axis=0, inplace=True)

aze_index = (final_df["country_name"] == "아제르바") & (final_df["continent_name"] == "Europe")
final_df.drop(final_df[aze_index].index, axis=0, inplace=True)

aru_index = (final_df["country_name"] == "아르메니아") & (final_df["continent_name"] == "Europe")
final_df.drop(final_df[aru_index].index, axis=0, inplace=True)

kf_index = (final_df["country_name"] == "키프로스") & (final_df["continent_name"] == "Asia")
final_df.drop(final_df[kf_index].index, axis=0, inplace=True)

aje_index = (final_df["country_name"] == "아제르바이잔") & (final_df["continent_name"] == "Europe")
final_df.drop(final_df[aje_index].index, axis=0, inplace=True)

In [52]:
final_df.duplicated().sum()

0

In [48]:
final_df.shape

(218, 10)

In [23]:
final_df.to_excel("preprocessed_df.xlsx", index=False)